In [1]:
import random
from collections import defaultdict

class FriendshipNetwork:
    def __init__(self, num_students=100, num_classes=4):
        self.num_students = num_students
        self.num_classes = num_classes
        self.students = list(range(num_students))
        self.classes = {i: [] for i in range(num_classes)}  # class_id -> list of students
        self.class_map = {}  # student -> class_id
        self.graph = defaultdict(list)  # adjacency list: student -> list of (friend, weight)

        self._assign_classes()
        self._generate_friendships()

    def _assign_classes(self):
        """Assign students to classes in a round-robin manner."""
        random.shuffle(self.students)
        for i, student in enumerate(self.students):
            class_id = i % self.num_classes
            self.classes[class_id].append(student)
            self.class_map[student] = class_id

    def _add_friendship(self, u, v, weight):
        """Add an undirected edge with weight between u and v."""
        self.graph[u].append((v, weight))
        self.graph[v].append((u, weight))

    def _generate_friendships(self):
        """Create friendships using probability rules for realism."""
        # Assign a 'popularity score' using an exponential distribution (a few are very popular)
        popularity = [random.expovariate(1 / 2) for _ in self.students]
        popularity = [p / max(popularity) for p in popularity]  # Normalize to 0–1

        for i in self.students:
            for j in self.students:
                if i >= j:
                    continue  # avoid self-loop and duplicate edges

                same_class = self.class_map[i] == self.class_map[j]
                pop_factor = (popularity[i] + popularity[j]) / 2

                # base friendship probability
                base_prob = 0.7 if same_class else 0.2

                # clustering boost: increase probability if they share a mutual friend
                neighbors_i = set(n for n, _ in self.graph[i])
                neighbors_j = set(n for n, _ in self.graph[j])
                if neighbors_i & neighbors_j:
                    base_prob += 0.2

                final_prob = base_prob * pop_factor

                if random.random() < final_prob:
                    weight = random.randint(1, 10)  # 1 = very close, 10 = distant friend
                    self._add_friendship(i, j, weight)

    def get_adjacency_list(self):
        return dict(self.graph)

    def print_sample(self, n=5):
        print(f"Showing {n} random students and their friends:")
        for student in random.sample(self.students, n):
            friends = self.graph[student]
            print(f"Student {student} (Class {self.class_map[student]}): {[f'{f} (w={w})' for f, w in friends]}")

    def get_class_map(self):
        return self.class_map


Random Sample:

In [2]:
if __name__ == "__main__":
    net = FriendshipNetwork(num_students=100, num_classes=4)
    net.print_sample()


Showing 5 random students and their friends:
Student 13 (Class 0): ['99 (w=4)', '48 (w=3)', '7 (w=2)']
Student 27 (Class 2): ['16 (w=8)', '71 (w=5)', '48 (w=10)']
Student 21 (Class 1): ['8 (w=10)', '72 (w=9)']
Student 60 (Class 0): ['28 (w=2)', '18 (w=2)', '31 (w=7)', '92 (w=10)', '94 (w=6)']
Student 40 (Class 2): ['76 (w=6)']


Connected(using BFS):

In [3]:
def bfs_connected_components(graph, nodes):
    visited = set()
    components = []

    for student in nodes:
        if student not in visited:
            queue = [student]
            group = []

            while queue:
                current = queue.pop(0)
                if current not in visited:
                    visited.add(current)
                    group.append(current)
                    neighbors = [nbr for nbr, _ in graph[current]]
                    queue.extend([n for n in neighbors if n not in visited])

            components.append(group)

    return components


Sample Output:

In [4]:
if __name__ == "__main__":
    net = FriendshipNetwork(num_students=100, num_classes=4)
    graph = net.get_adjacency_list()
    students = list(range(100))

    components = bfs_connected_components(graph, students)

    print(f"Number of friend groups: {len(components)}")
    group_sizes = [len(group) for group in components]
    print(f"Smallest group size: {min(group_sizes)}")
    print(f"Largest group size: {max(group_sizes)}")


Number of friend groups: 1
Smallest group size: 100
Largest group size: 100


Dijkstra's shortest path:

In [5]:
def dijkstra(graph, start, goal):
    heap = [(0, start, [])]
    visited = set()

    while heap:
        cost, node, path = heapq.heappop(heap)

        if node in visited:
            continue
        visited.add(node)
        path = path + [node]

        if node == goal:
            return cost, path

        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(heap, (cost + weight, neighbor, path))

    return float('inf'), []


A*'s shortest path:

In [6]:
def astar(graph, start, goal, heuristic_func):
    heap = [(0, 0, start, [])]  # f, g, node, path
    visited = set()

    while heap:
        f, g, node, path = heapq.heappop(heap)

        if node in visited:
            continue
        visited.add(node)
        path = path + [node]

        if node == goal:
            return g, path

        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                h = heuristic_func(neighbor, goal)
                heapq.heappush(heap, (g + weight + h, g + weight, neighbor, path))

    return float('inf'), []


Calling heuristic function in A* :

In [7]:
def simple_class_heuristic(u, v, class_map):
    return 0 if class_map[u] == class_map[v] else 5

Comparison:

In [8]:
if __name__ == "__main__":
    net = FriendshipNetwork(num_students=100, num_classes=4)
    graph = net.get_adjacency_list()
    class_map = net.get_class_map()
    students = list(graph.keys())

    # Select 5 random unique student pairs
    pairs = []
    while len(pairs) < 5:
        a, b = random.sample(students, 2)
        if (a, b) not in pairs and (b, a) not in pairs:
            pairs.append((a, b))

    print("Shortest Friendship Paths (Dijkstra):")
    for i, (u, v) in enumerate(pairs):
        cost, path = dijkstra(graph, u, v)
        print(f"Pair {i+1}: {u} ➝ {v} | Cost: {cost} | Path: {path}")

    # Apply A* to one pair
    u, v = pairs[0]
    cost_astar, path_astar = astar(graph, u, v, lambda x, y: simple_class_heuristic(x, y, class_map))

    print(f"\nA* Algorithm (Pair 1: {u} ➝ {v})")
    print(f"A* Cost: {cost_astar} | Path: {path_astar}")

    cost_dijk, path_dijk = dijkstra(graph, u, v)
    print(f"Dijkstra Cost: {cost_dijk} | Path: {path_dijk}")


Shortest Friendship Paths (Dijkstra):


NameError: name 'heapq' is not defined

In [ ]:
from collections import deque

def bfs_connected_components(graph, nodes):
    visited = set()
    components = []

    for node in nodes:
        if node not in visited:
            component = []
            queue = deque([node])
            visited.add(node)

            while queue:
                current = queue.popleft()
                component.append(current)

                # Iterate neighbors
                for neighbor, _weight in graph.get(current, []):
                    if neighbor in nodes and neighbor not in visited:
                        visited.add(neighbor)
                        queue.append(neighbor)

            components.append(component)

    return components


def detect_bridge_nodes(graph, students):
    original_components = bfs_connected_components(graph, students)
    original_count = len(original_components)

    bridge_nodes = []

    for student in students:
        # Copy graph excluding the student
        modified_graph = {
            node: [(nbr, w) for nbr, w in nbrs if nbr != student]
            for node, nbrs in graph.items() if node != student
        }

        # Nodes remaining after removal
        remaining_nodes = [n for n in students if n != student]
        new_components = bfs_connected_components(modified_graph, remaining_nodes)

        if len(new_components) > original_count:
            bridge_nodes.append(student)

    return bridge_nodes


In [ ]:
if __name__ == "__main__":
    net = FriendshipNetwork(num_students=100, num_classes=4)
    graph = net.get_adjacency_list()
    students = list(range(100))

    bridge_nodes = detect_bridge_nodes(graph, students)

    print("Bridge Nodes Detected:")
    print(bridge_nodes)


Bridge Nodes Detected:
[58]
